

# Carga data



In [ ]:
%pip install langchain-elasticsearch langchain-community langchain tiktoken langchain_openai pdfplumber pypdf


In [ ]:
import os
with open("/content/api_key.txt") as archivo:
  apikey = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore
from langchain_community.document_loaders import PyPDFLoader
import re
from typing import List, Dict
from langchain.schema import Document

In [ ]:
from typing import List, Dict
from langchain.schema import Document

def procesar_pdf_con_metadata(ruta_pdf: str, category: str, extra_metadata: Dict = None) -> List[Document]:
    """
    Carga un PDF, limpia el texto de cada chunk, aplica metadatos y devuelve una lista de Documents.

    Args:
        ruta_pdf (str): Ruta al archivo PDF.
        category (str): Categoría principal (e.g., 'deporte', 'nutricion').
        extra_metadata (dict): Otros metadatos adicionales (opcional).

    Returns:
        List[Document]: Lista de documentos listos para dividir o subir a un vector store.
    """
    custom_metadata = {"category": category}
    if extra_metadata:
        custom_metadata.update(extra_metadata)

    # Cargar PDF y dividir por páginas
    loader = PyPDFLoader(ruta_pdf)
    docs = loader.load_and_split()

    # Limpiar texto en cada página
    for doc in docs:
        text = doc.page_content
        text = re.sub(r'\n{2,}', '\n\n', text)  # Mantiene saltos de párrafo
        text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)  # Une líneas fragmentadas
        text = re.sub(r'\.{2,}', '', text)                # Elimina múltiples puntos consecutivos

        doc.page_content = text

        # Agregar metadatos personalizados

        doc.metadata.update(custom_metadata)

    return docs


In [ ]:
docs_deporte

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'creationdate': '2019-04-22T15:28:19+02:00', 'trapped': '/False', 'moddate': '2019-04-22T15:29:06+02:00', 'source': '/content/documento.pdf', 'total_pages': 347, 'page': 0, 'page_label': '1', 'category': 'deporte'}, page_content='SEGUNDA EDICIÓN THE MUSCLE  & STRENGTH  PYRAMID ENTRENAMIENTO Eric Helms, PhD, CSCS Andrea Valdez, MS Andy Morgan, BS José Carlos De Francisco Víctor Reyes Alberto Alvarez Adaptado y traducido por: Preparado en exclusiva para Jesús Gustavo Aguirre Valenciana, jgustavo.avalenciana1@hotmail.com #13299859'),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'creationdate': '2019-04-22T15:28:19+02:00', 'trapped': '/False', 'moddate': '2019-04-22T15:29:06+02:00', 'source': '/content/documento.pdf', 'total_pages': 347, 'page': 1, 'page_label': '2', 'category': 'deporte'}, page_content='Este libro no está diseña

In [ ]:
docs_deporte = procesar_pdf_con_metadata(
    ruta_pdf="/content/documento.pdf",
    category="deporte",
    extra_metadata={"category": "deporte"}
)
docs_deporte[:2]

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'creationdate': '2019-04-22T15:28:19+02:00', 'trapped': '/False', 'moddate': '2019-04-22T15:29:06+02:00', 'source': '/content/documento.pdf', 'total_pages': 347, 'page': 0, 'page_label': '1', 'category': 'deporte'}, page_content='SEGUNDA EDICIÓN THE MUSCLE  & STRENGTH  PYRAMID ENTRENAMIENTO Eric Helms, PhD, CSCS Andrea Valdez, MS Andy Morgan, BS José Carlos De Francisco Víctor Reyes Alberto Alvarez Adaptado y traducido por: Preparado en exclusiva para Jesús Gustavo Aguirre Valenciana, jgustavo.avalenciana1@hotmail.com #13299859'),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'creationdate': '2019-04-22T15:28:19+02:00', 'trapped': '/False', 'moddate': '2019-04-22T15:29:06+02:00', 'source': '/content/documento.pdf', 'total_pages': 347, 'page': 1, 'page_label': '2', 'category': 'deporte'}, page_content='Este libro no está diseña

In [ ]:
#Dividir los documentos
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(docs_deporte)
print(docs[:3])  # Mostrar los primeros tres fragmentos

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'creationdate': '2019-04-22T15:28:19+02:00', 'trapped': '/False', 'moddate': '2019-04-22T15:29:06+02:00', 'source': '/content/documento.pdf', 'total_pages': 347, 'page': 0, 'page_label': '1', 'category': 'deporte'}, page_content='SEGUNDA EDICIÓN THE MUSCLE  & STRENGTH  PYRAMID ENTRENAMIENTO Eric Helms, PhD, CSCS Andrea Valdez, MS Andy Morgan, BS José Carlos De Francisco Víctor Reyes Alberto Alvarez Adaptado y traducido por: Preparado en exclusiva para Jesús Gustavo Aguirre Valenciana, jgustavo.avalenciana1@hotmail.com #13299859'), Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'creationdate': '2019-04-22T15:28:19+02:00', 'trapped': '/False', 'moddate': '2019-04-22T15:29:06+02:00', 'source': '/content/documento.pdf', 'total_pages': 347, 'page': 1, 'page_label': '2', 'category': 'deporte'}, page_content='Este libro no está diseñad

In [ ]:
docs[:3]

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'creationdate': '2019-04-22T15:28:19+02:00', 'trapped': '/False', 'moddate': '2019-04-22T15:29:06+02:00', 'source': '/content/documento.pdf', 'total_pages': 347, 'page': 0, 'page_label': '1', 'category': 'deporte'}, page_content='SEGUNDA EDICIÓN THE MUSCLE  & STRENGTH  PYRAMID ENTRENAMIENTO Eric Helms, PhD, CSCS Andrea Valdez, MS Andy Morgan, BS José Carlos De Francisco Víctor Reyes Alberto Alvarez Adaptado y traducido por: Preparado en exclusiva para Jesús Gustavo Aguirre Valenciana, jgustavo.avalenciana1@hotmail.com #13299859'),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Macintosh)', 'creationdate': '2019-04-22T15:28:19+02:00', 'trapped': '/False', 'moddate': '2019-04-22T15:29:06+02:00', 'source': '/content/documento.pdf', 'total_pages': 347, 'page': 1, 'page_label': '2', 'category': 'deporte'}, page_content='Este libro no está diseña

In [ ]:
#Generar embeddigs
embeddings = OpenAIEmbeddings()

db = ElasticsearchStore.from_documents(
    docs,
    embeddings,
    es_url="http://34.16.182.210:9200",
    es_user="elastic",
    es_password="BCQ0UvQC7yxHCXNMLoU_",
    index_name="nutricion_y_deporte-data",
)

In [ ]:
db.client.indices.refresh(index="nutricion_y_deporte-data")


ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})